### SE module


In [1]:
import math
import torch.nn as nn
import torch
import numpy

dim_in = 24
dim_mid = 24

kernel_reduce = torch.zeros(dim_mid, dim_in, 1, 1)
for k in range(dim_mid):
    for l in range(dim_in):
        for i in range(1):
            for j in range(1):
                kernel_reduce[k, l, i, j] = 0.1

kernel_expand = torch.zeros(dim_in, dim_mid, 1, 1)
for k in range(dim_in):
    for l in range(dim_mid):
        for i in range(1):
            for j in range(1):
                kernel_expand[k, l, i, j] = 0.005

bias_reduce = torch.zeros(24)
bias_expand = torch.zeros(24)
for i in range(dim_mid):
    bias_reduce[i] = i + 0.01
    bias_expand[i] = i + 0.01
# 實際上只做一次SE, make_divisible
# divisor=1
# 基本上這個東西沒做事(?)若se_ratio不為1，傳進來的v為小數，return new_v會處理成整數
# 目前看起來re_ratio!=se_ratio，則se_ratio為SE預設的0.25
def make_divisible(v, divisor=1):
    min_value = 1
    new_v = max(min_value, int(v + divisor / 2))
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        print('too small', new_v, v)
        new_v += divisor
    return new_v


def SE(dim_in, ratio, x):
    print('input data', x.shape)
    reduced_chs = int(dim_in * ratio)
    # self.conv = nn.Conv2d(dim_in, dim_out, kernel_size, stride, padding, dilation, groups, bias)
    conv_reduce = nn.Conv2d(dim_in, reduced_chs, 1, bias=True)
    #print('reduce kernel', conv_reduce.weight.data.shape)
    conv_reduce.weight.data = kernel_reduce
    conv_reduce.bias.data = bias_reduce
    conv_expand = nn.Conv2d(reduced_chs, dim_in, 1, bias=True)
    #print('expand kernel', conv_expand.weight.shape)
    conv_expand.weight.data = kernel_expand
    conv_expand.bias.data = bias_expand

    x_se = x.mean((2, 3), keepdim=True)  # (0, 1, 2, 3) => (b, c, h, w)
    print('x_se mean')
    print(x_se)
    x_se = conv_reduce(x_se)
    print('x_se reduce')
    print(x_se)
    x_se = torch.relu(x_se)
    x_se = conv_expand(x_se)

    return x * torch.sigmoid(x_se) 


In [2]:
import torch

batch_size = 1
dim_in = 24
image_size = 8
ratio = 1  # stage1-4: 4

# 生成隨機的模擬圖像數據，使用 PyTorch 的隨機函數
test_images = torch.zeros(batch_size, dim_in, image_size, image_size)
for b in range(batch_size):
    for c in range(dim_in):
        for i in range(image_size):
            for j in range(image_size):
                test_images[b, c, i, j] = c + i

# 將數據傳遞給 SE 函數進行測試
x_se = SE(dim_in, ratio, test_images)

# 輸出測試結果
print(x_se.shape)
print(x_se[0, 0, :, :])


input data torch.Size([1, 24, 8, 8])
x_se mean
tensor([[[[ 3.5000]],

         [[ 4.5000]],

         [[ 5.5000]],

         [[ 6.5000]],

         [[ 7.5000]],

         [[ 8.5000]],

         [[ 9.5000]],

         [[10.5000]],

         [[11.5000]],

         [[12.5000]],

         [[13.5000]],

         [[14.5000]],

         [[15.5000]],

         [[16.5000]],

         [[17.5000]],

         [[18.5000]],

         [[19.5000]],

         [[20.5000]],

         [[21.5000]],

         [[22.5000]],

         [[23.5000]],

         [[24.5000]],

         [[25.5000]],

         [[26.5000]]]])
x_se reduce
tensor([[[[36.0100]],

         [[37.0100]],

         [[38.0100]],

         [[39.0100]],

         [[40.0100]],

         [[41.0100]],

         [[42.0100]],

         [[43.0100]],

         [[44.0100]],

         [[45.0100]],

         [[46.0100]],

         [[47.0100]],

         [[48.0100]],

         [[49.0100]],

         [[50.0100]],

         [[51.0100]],

         [[52.0100]]